In [1]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

import torchvision.transforms as transforms
import torchvision
import torch.optim as optim
import torch.nn as nn
import pandas as pd
import numpy as np
from datetime import datetime

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

In [2]:
label_frame = pd.read_csv('LabelTrain.csv')

In [3]:
label_frame.head()

,img,label
0,10157840@N03_identity_0@3909066374_1.jpg,0
1,10175970@N00_identity_38@239770954_1.jpg,0
2,10175970@N00_identity_38@334572069_0.jpg,0
3,10175970@N00_identity_38@334574437_1.jpg,0
4,10175970@N00_identity_38@334599066_0.jpg,0


In [4]:
class Dataset(Dataset):

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.lable_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.lable_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.lable_frame.iloc[idx, 0])
        image = io.imread(img_name)
        label = self.lable_frame.iloc[idx, 1]
        label = np.array(label)
        label = label.astype('int').reshape(1)
        sample = {'image': image, 'label': label}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [5]:
face_dataset = Dataset(csv_file='LabelTrain.csv',
                        root_dir=r'C:\Users\11507\Desktop\X Academy 2021\MeGlass_120x120\MeGlass_120x120\Without/Train')

for i in range(len(face_dataset)):
    sample = face_dataset[i]
    print(i, sample['image'].shape, sample['label'].shape)
    if i == 5:
        break

0 (120, 120, 3) (1,)
1 (120, 120, 3) (1,)
2 (120, 120, 3) (1,)
3 (120, 120, 3) (1,)
4 (120, 120, 3) (1,)
5 (120, 120, 3) (1,)


In [6]:
val_dataset = Dataset(csv_file='LabelVal.csv',
                        root_dir=r'C:\Users\11507\Desktop\X Academy 2021\MeGlass_120x120\MeGlass_120x120\Without/Train')

for i in range(len(face_dataset)):
    sample = face_dataset[i]
    print(i, sample['image'].shape, sample['label'].shape)
    if i == 5:
        break

0 (120, 120, 3) (1,)
1 (120, 120, 3) (1,)
2 (120, 120, 3) (1,)
3 (120, 120, 3) (1,)
4 (120, 120, 3) (1,)
5 (120, 120, 3) (1,)


In [7]:
# tsfm = Transform(params)
# transformed_sample = tsfm(sample)

In [8]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""
    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C x H x W
        image = image.transpose((2, 0, 1))
        return {'image': torch.from_numpy(image),
                'label': torch.from_numpy(label)}


In [9]:
import torchvision.transforms as transforms

In [10]:
transformed_dataset = Dataset(csv_file='LabelTrain.csv',
                        root_dir=r'C:\Users\11507\Desktop\X Academy 2021\MeGlass_120x120\MeGlass_120x120\Without/Train',
                        transform=ToTensor())
for i in range(len(transformed_dataset)):
    sample = transformed_dataset[i]

    print(i, sample['image'].size(), sample['label'].size())

    if i == 3:
        break

0 torch.Size([3, 120, 120]) torch.Size([1])
1 torch.Size([3, 120, 120]) torch.Size([1])
2 torch.Size([3, 120, 120]) torch.Size([1])
3 torch.Size([3, 120, 120]) torch.Size([1])


In [11]:
transformedval_dataset = Dataset(csv_file='LabelVal.csv',
                        root_dir=r'C:\Users\11507\Desktop\X Academy 2021\MeGlass_120x120\MeGlass_120x120\Without/validation',
                        transform=ToTensor())
for i in range(len(transformedval_dataset)):
    sample = transformedval_dataset[i]

    print(i, sample['image'].size(), sample['label'].size())

    if i == 3:
        break
print(len(transformedval_dataset))

0 torch.Size([3, 120, 120]) torch.Size([1])
1 torch.Size([3, 120, 120]) torch.Size([1])
2 torch.Size([3, 120, 120]) torch.Size([1])
3 torch.Size([3, 120, 120]) torch.Size([1])
2511


In [12]:
trainset = transformed_dataset
trainloader = DataLoader(trainset , batch_size=64,
                        shuffle=True, num_workers=0)
testset = transformedval_dataset
testloader = DataLoader(testset , batch_size=32,
                        shuffle=False, num_workers=0)
classes = ('big', 'small')

In [13]:
if torch.cuda.is_available():
    print('using GPU, all good')
    device = torch.device('cuda:0')
else:
    print('using CPU, if you are in colab, please change the '
          'setting in Runtime/changeRuntimeType to GPU')
    device = torch.device('cpu')

using CPU, if you are in colab, please change the setting in Runtime/changeRuntimeType to GPU


In [14]:
class yoyoBSModule(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 96, 11, padding=5)
        self.conv2 = nn.Conv2d(96, 256, 5, padding=2)
        self.pool = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(256, 384, 3, padding=1)
        self.conv4 = nn.Conv2d(384, 384, 3, padding=1)
        self.conv5 = nn.Conv2d(384, 256, 3, padding=1)
        self.fc6 = nn.Linear(4096, 4096)
        self.fc7 = nn.Linear(4096, 1000)
        self.fc8 = nn.Linear(1000, 2)
        self.flatten = nn.Flatten()
        self.softmax = nn.Softmax(dim =1)
        self.ReLU = nn.ReLU()

    def forward(self, x):
        # x shape (B, 3, 120, 120)
        x = self.conv1(x) # (B, 96, 120, 120)
        x = self.ReLU(x)
        x = self.conv2(x) # (B, 256, 120, 120)
        x = self.ReLU(x)
        x = self.pool(x) # (B, 256, 60, 60)
        x = self.conv3(x)
        x = self.ReLU(x)
        x = self.pool(x)
        x = self.conv4(x)
        x = self.ReLU(x)
        x = self.conv5(x)
        x = self.ReLU(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.fc6(x)
        x = self.ReLU(x)
        x = self.fc7(x)
        x = self.ReLU(x)
        x = self.fc8(x)
        return x

In [15]:
for i, data in enumerate(trainloader, 0):
    input, labels = data['image'], data['label']
   # print(labels)

In [ ]:
def train(args, model, train_loader, test_loader):
    print(f'now training with {args}')
    model = model.to(device)
    optimizer = optim.SGD(model.parameters(), lr=args['learning_rate'],
                          momentum=0.9, weight_decay=args['l2'])
    criterion = nn.CrossEntropyLoss()

    df = pd.DataFrame()
    last_time = datetime.now()
    for epoch in range(args['total_epoch']):
        training_loss = test_loss = 0
        training_acc = test_acc = 0
        model.train()
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            # move the tensors to gpu if we are using it
            # data loading happens in cpu so the tensors were there initially
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # calculate stats
            training_loss += loss.item()
            _, prediceted = torch.max(outputs.data, 1)
            training_acc += (prediceted==labels).sum().item()

        # evaluate the model on validation set

        with torch.no_grad():
            model.eval()
            for i, data in enumerate(testloader, 0):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                # calculate stats
                test_loss += loss.item()
                _, prediceted = torch.max(outputs.data, 1)
                test_acc += (prediceted == labels).sum().item()

        training_loss /= len(trainloader)
        training_acc /= len(trainset)
        test_loss /= len(testloader)
        test_acc /= len(testset)
        data = {
            'epoch':epoch,
            'training_loss': training_loss,
            'training_acc': training_acc,
            'test_loss': test_loss,
            'test_acc':test_acc,
            'time': (datetime.now()-last_time).total_seconds()
        }
        print(data)
        df = df.append(data, ignore_index=True)
        last_time = datetime.now()
    return model, df



In [ ]:
args = {
    'learning_rate': 1e-2,
    'total_epoch': 12,
    'l2': 0,
}

# run it!
model = yoyoBSModule()
model, data = train(args, model, trainloader, testloader)
data.plot('epoch', ['training_loss', 'test_loss'])